In [3]:
# Importing the libraries and removing the warning messages

import pandas as pd
import numpy as np
import warnings
import sys
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib import ticker
from statsmodels.tsa.stattools import adfuller, acf, pacf
from sklearn.preprocessing import MinMaxScaler

warnings.filterwarnings('ignore')

In [4]:
# Importing the scripts

sys.path.append("..")
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.ensemble import RandomForestRegressor

In [5]:
# Importing and reading the data

train = pd.read_csv('../data/train_data_clean.csv',index_col='Date')
train.info()